![AIAP Banner](../images/AIAP-Banner.png "AIAP Banner")

<h1><center>Assignment 1 - Part 1: 
<br>
Data Cleaning & Feature Engineering</center></h1>

<h3>Name: Wong Khee Ern</h3>

---

# Before starting...

### How to structure your answers

There are many different ways to approach a problem in AI/ML. When attempting the problems in the notebooks, keep in mind that there is no definite "correct way" to handle any problem. Instead, when choosing which method to use, focus on how current literature approaches the problem, how you intend to evaluate the effectiveness of the solution you are proposing and how to improve the solution if necessary. Similarly, for the "open-ended" questions, it is important to substantiate your answers with supporting reasons. Be sure to bounce your ideas off each other to see how different people approach the same problem!

### Coding conventions

Learning good habits and ensuring your code follows a certain convention is very important in an environment where your code will be shared and read by others. Following the standard conventions highlighted in the [PEP-8 document](https://www.python.org/dev/peps/pep-0008/) is a good start.

### Reproducible Data Pipeline

In the next few sections, you will be creating a data pipeline in a step by step process. At the end of this notebook, you have to combine all of these steps into a Python module named `datapipeline.py` in the [src folder](./src). The module should contain at least a function with the following signature:

```python
def transform(data_path):
  """
  :param data_path: ......
  :return: ......
  """
  return feature_engineered_dataframe
```

Once complete, you will be able to use this function freely in this notebook or other notebooks. This will ensure consistency in the data transformation process.

# 1. Introduction

Most machine learning projects follow a typical flow. 

    - Defining the problem statement
    - Identifying an appropriate dataset
    - Extracting the relevant data from data sources
    - Defining and labelling the dataset
    - Data cleaning
    - Exploring the dataset, refered to as Exploratory Data Analysis (EDA)
    - Feature engineering
    - Selecting and training appropriate models
    - Evaluating the model
    - Repeating the above steps (data identification to model evaluation) until desired performance is achieved
    - Deploying the model
    - Model maintenance and retraining (if necessary)


For this assignment, we will explore clustering in unsupervised learning. Keep this in mind when performing the data preparation steps (data extraction, data cleaning, EDA and feature engineering) in the next sections of this notebook.

#### 1.1. Topics
1. SQL query
2. Data cleaning
3. Exploratory data analysis
4. Feature Engineering
5. Data pipeline

#### 1.2. Deliverables
1. Jupyter notebook
2. Script: `datapipeline.py`

# 2. Data Extraction

The data for this assignment were modified from the US census-income dataset on the UCI dataset repository. **Please do not download the data directly from the UCI repository. Instead, follow the instructions below to query the dataset from our database.** Additionally, please refer to this [link](https://www2.1010data.com/documentationcenter/beta/Tutorials/MachineLearningExamples/CensusIncomeDataSet.html) for a description of the variables.

The data are stored in a database. If you are not familiar with databases, please refer to this [link](https://medium.com/@rwilliams_bv/intro-to-databases-for-people-who-dont-know-a-whole-lot-about-them-a64ae9af712) to obtain a high level overview of databases and their related terms.

The type of database used is an Azure SQL Server instance. SQL Server is Microsoft's RDBMS product offering which uses a variant of SQL (Structured Query Language) called T-SQL (Microsoft Transact-SQL).
There are many resources available online to learn how to write T-SQL and you should be able to find one that fits to your level of understanding. One such resource is on [TutorialsPoint](https://www.tutorialspoint.com/t_sql/index.htm). In addition, you can use Microsoft's [reference pages](https://docs.microsoft.com/en-us/sql/t-sql/language-reference?view=sql-server-ver15) to quickly look up syntax documentation.
At a minimum, you should be able to combine and extract data from multiple tables in an efficient manner so that you can complete your assignments.

If you have trouble accessing the database, you may have to download the Microsoft ODBC Driver. Follow the download instructions for [Windows](https://docs.microsoft.com/en-us/sql/connect/odbc/windows/system-requirements-installation-and-driver-files?view=sql-server-ver15) or [Mac/Linux](https://docs.microsoft.com/en-us/sql/connect/odbc/linux-mac/installing-the-microsoft-odbc-driver-for-sql-server?view=sql-server-ver15#microsoft-odbc-driver-131-for-sql-server).

The data is hosted on an Azure SQL Server with the following details:

    server = 'aiap-training.database.windows.net'
    database = 'aiap'
    username = 'apprentice'
    password = 'Pa55w.rd'
    driver= '{ODBC Driver 17 for SQL Server}'


There are 3 separate tables, `basic_bio`, `employment`, `residential_tax`. All of these tables have an `id` column that can be used to merge them.


The pandas package has a `read_sql_query` function that can be used to access the data. You may require another python package to use this function.

#### 2.1. Query all 3 tables from the SQL server and combine them into a single pandas dataframe. Save this dataframe as a `.csv` file on your local computer with the filepath `assignment1/data/raw/data.csv`.

In [1]:
# Import libraries
import pandas as pd
import os
from sqlalchemy import create_engine
import urllib.parse

In [6]:
# Database connection details
server = 'aiap-training.database.windows.net'
database = 'aiap'
username = 'apprentice'
password = 'Pa55w.rd'

# URL encode the password
password_encoded = urllib.parse.quote_plus(password)

# Create SQLAlchemy URL format connection string
connection_string = f"mssql+pyodbc://{username}:{password_encoded}@{server}/{database}?driver=ODBC+Driver+18+for+SQL+Server&TrustServerCertificate=yes&Connection+Timeout=60&Encrypt=yes"

# Create SQLAlchemy engine
engine = create_engine(connection_string)

# Query all 3 tables
query = """
SELECT *
FROM basic_bio b
INNER JOIN employment e ON b.id = e.id
INNER JOIN residential_tax r ON b.id = r.id
"""

df = pd.read_sql_query(query, engine)

In [7]:
# Save the data (Jupyter uses current dir as the 'root')
os.makedirs('data/raw', exist_ok=True)
df.to_csv('data/raw/data.csv', index=False)

print("✅ File saved: assignment1/data/raw/data.csv")

✅ File saved: assignment1/data/raw/data.csv


# 3. Data Cleaning

Whenever we have a new dataset, we need to inspect and clean the dataset.

Some (non-exhaustive) steps in data cleaning are:
    - handling missing values
    - checking for irrelevant rows and/or columns
    - checking for duplicate rows and deciding whether to drop them
    
    
#### 3.1. List the steps you intend to take to clean the data in the markdown cell below. Give a brief explanation for each step.

Inspect data
1. Find out the shape of the data
2. Find out what the columns mean and its data type
3. Inspect each column individually for: (use plots to visualize if needed)
    a. Missing values (msno, )
    b. Outliers (describe(), IQR, box plots)
    c. Distribution (histogram, countplots etc.)
4. Check for duplicates
5. Check for correlation, effect sizes between different columns (corr matrix, heatmap)

Clean data
1. For each column:
    a. Missing values
        i. Check if missing values are correlated with other columns
        ii. Decide on the best way to fill in missing values
            - Ideally, try to find patterns in data to fill in missing values
            - Else, try mean, mode, median imputation
            - If too many missing values, can consider dropping the col
    b. Outliers
        i. Find out more domain knowledge to see if outliers are valid or simply data errors
        ii. Analyze their potential impact on the ML model
        iii. Identify methods to address outliers (log, winsorizing etc, knn etc.)
    c. Others
        i. Spelling errors, data type and formatting inconsistencies, formatting date time, rules violation (e.g. negative age)
2. Drop duplicate data where necessary by applying domain knowledge



#### 3.2. Write each data cleaning step as its own function so that they can be reused individually. Ideally, you should organise the functions so that they can be put into their own library. Remember to follow the [PEP8](https://www.python.org/dev/peps/pep-0008/) convention.

In [11]:
for col in df.columns:  
    print(f"Column '{col}': type = {type(df[col])}")

Column 'id': type = <class 'pandas.core.frame.DataFrame'>
Column 'age': type = <class 'pandas.core.frame.DataFrame'>
Column 'education': type = <class 'pandas.core.frame.DataFrame'>
Column 'enroll_in_edu_inst_last_wk': type = <class 'pandas.core.series.Series'>
Column 'marital_stat': type = <class 'pandas.core.frame.DataFrame'>
Column 'race': type = <class 'pandas.core.frame.DataFrame'>
Column 'sex': type = <class 'pandas.core.frame.DataFrame'>
Column 'country_of_birth_father': type = <class 'pandas.core.series.Series'>
Column 'country_of_birth_mother': type = <class 'pandas.core.series.Series'>
Column 'country_of_birth_self': type = <class 'pandas.core.series.Series'>
Column 'citizenship': type = <class 'pandas.core.frame.DataFrame'>
Column 'id': type = <class 'pandas.core.frame.DataFrame'>
Column 'age': type = <class 'pandas.core.frame.DataFrame'>
Column 'class_of_worker': type = <class 'pandas.core.series.Series'>
Column 'detailed_industry_recode': type = <class 'pandas.core.series.

In [13]:
df.shape

(199463, 50)

In [12]:
def data_overview(df, name="Dataset"):
   """
   Comprehensive data overview for exploratory data analysis.
   
   Parameters:
   -----------
   df : pandas.DataFrame
       The dataset to analyze
   name : str, optional
       Name of the dataset for display purposes (default: "Dataset")
   
   Returns:
   --------
   dict
       Dictionary containing overview statistics for programmatic use
   """
   
   print(f"{'='*60}")
   print(f"DATA OVERVIEW: {name}")
   print(f"{'='*60}")
   
   # Basic shape information
   n_rows, n_cols = df.shape
   print(f"Shape: {n_rows:,} rows × {n_cols} columns")
   
   # Memory usage
   memory_mb = df.memory_usage(deep=True).sum() / (1024 * 1024)
   print(f"Memory Usage: {memory_mb:.2f} MB")
   
   # Data types breakdown
   print("\nData Types:")
   dtype_counts = df.dtypes.value_counts()
   for dtype, count in dtype_counts.items():
       print(f"  {str(dtype):<12}: {count} columns")
   
   # Column overview
   print()
   print(f"{'Column':<25} {'Type':<15} {'Non-Null':<10} {'Null%':<8} {'Unique':<8}")
   print("-" * 70)
   
   overview_stats = {}
   
   for col in df.columns:
       col_type = str(df[col].dtype)
       non_null_count = df[col].count()
       null_pct = ((n_rows - non_null_count) / n_rows * 100)
       unique_count = df[col].nunique()
       
       print(f"{col[:24]:<25} {col_type:<15} {non_null_count:<10} "
             f"{null_pct:>6.1f}% {unique_count:<8}")
       
       # Store for programmatic use
       overview_stats[col] = {
           'dtype': col_type,
           'non_null_count': non_null_count,
           'null_percentage': null_pct,
           'unique_count': unique_count
       }
   
   # Summary statistics
   missing_cols = df.isnull().sum()
   high_missing = missing_cols[missing_cols > n_rows * 0.5]
   
   print(f"\n{'='*30}")
   print("SUMMARY")
   print(f"{'='*30}")
   print(f"Total missing values: {df.isnull().sum().sum():,}")
   print(f"Columns with >50% missing: {len(high_missing)}")
   
   if len(high_missing) > 0:
       print("High missing columns:", list(high_missing.index))
   
   # Potential issues
   print("\nPOTENTIAL ISSUES:")
   duplicate_rows = df.duplicated().sum()
   print("• Duplicate rows: {duplicate_rows:,}")
   
   # Check for columns that might be IDs
   potential_ids = []
   for col in df.columns:
       if df[col].nunique() == n_rows and n_rows > 1:
           potential_ids.append(col)
   
   if potential_ids:
       print(f"• Potential ID columns: {potential_ids}")
   
   # Return summary for programmatic use
   return {
       'shape': (n_rows, n_cols),
       'memory_mb': memory_mb,
       'dtype_counts': dtype_counts.to_dict(),
       'column_stats': overview_stats,
       'total_missing': df.isnull().sum().sum(),
       'duplicate_rows': duplicate_rows,
       'potential_id_columns': potential_ids
   }


data_overview(df)
# Example usage:
# stats = data_overview(df, name="Sales Dataset")
# print(f"Dataset has {stats['duplicate_rows']} duplicate rows")

DATA OVERVIEW: Dataset
Shape: 199,463 rows × 50 columns
Memory Usage: 436.10 MB

Data Types:
  object      : 34 columns
  int64       : 16 columns

Column                    Type            Non-Null   Null%    Unique  
----------------------------------------------------------------------


AttributeError: 'DataFrame' object has no attribute 'dtype'

# 4. Exploratory Data Analysis

After the data has been cleaned, we now need to perform EDA to get a better understanding of the dataset. As a start, you can consider looking at the distributions of each variable as well as the correlation between each pair of variables. Please explore further if you find any meaningful insights. You can refer to this [book chapter](https://www.stat.cmu.edu/~hseltman/309/Book/chapter4.pdf) for detailed information on EDA.


#### 4.1. Perform EDA on the dataset. Include all figures / statistics you use. Briefly describe the purpose for each EDA step and the finding(s) from each step.

# 5. Feature Engineering

Feature engineering is the process of transforming variables so that they are potentially more useful for the task at hand. Domain expertise is extremely useful for feature engineering. Hence, it is always important to try and understand as much as you can about the problem's domain.

Apart from using domain knowledge, we can also perform some simple feature engineering by aggregating different categories within a variable.

When conducting feature engineering, it is important to keep in mind the task at hand and take reference from the insights that were derived from the EDA. This will ensure that the features that were created will be useful for the task at hand. The main task that you are feature engineering for is unsupervised learning (clustering).

#### 5.1. Engineer 3 new features for this dataset. Briefly explain why you chose these features and how you think they could be useful for clustering.

# 6. Reproducible Data Pipeline

#### 6.1. Finally, take all the code you've written and create a Python module named `datapipeline.py` in the [src folder](./src). The module should at least contain the following function. The transform function should take in the absolute path to the data file as a parameter and return a pandas dataframe.

```python
def transform(data_path):
  """
  :param data_path: ......
  :return: ......
  """
  return feature_engineered_dataframe
```

# 7. Submission


#### 7.1. Create a `conda.yml` file at the base of the assignment folder. Add (manually) your required dependencies to the file named `conda.yml` .

# Explore Clustering Algorithms

Load the '[`A1P2_clustering.ipynb`](A1P2_clustering.ipynb)' notebook and start working from there.

<h1><center>End of Assignment 1 - Part 1: Data Cleaning & Feature Engineering</center></h1>